In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset

In [119]:
pd.set_option('display.max_columns', None)

In [120]:
df = pd.read_csv('academic_train.csv',header=0,index_col=0)
df.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,9238,1,1,126.0,1,1,19,5,5,122.6,0,0,0,1,0,1,18,0,0,6,6,6,14.500000,0,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,17,1,9238,1,1,125.0,1,19,19,9,9,119.8,1,0,0,1,0,0,18,0,0,6,8,4,11.600000,0,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,1,17,2,9254,1,1,137.0,1,3,19,2,3,144.7,0,0,0,1,1,0,18,0,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,1,1,3,9500,1,1,131.0,1,19,3,3,2,126.1,1,0,0,1,0,1,18,0,0,7,9,7,12.591250,0,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,1,1,2,9500,1,1,132.0,1,19,37,4,9,120.1,1,0,0,1,0,0,18,0,0,7,12,6,12.933333,0,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [121]:
print(df.shape)
print(df.isna().sum().sum())

(76518, 37)
0


In [122]:
x,y = df.iloc[:,:df.shape[1]-1],df.iloc[:,-1]

In [123]:
make_dummy_cols = [1,2,3,7,8,9,10,11]
standardize_cols = [6,12,19,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35]
make_dummy_cols = [df.columns[i] for i in make_dummy_cols]
standardize_cols = [df.columns[i] for i in standardize_cols]

In [124]:
x[standardize_cols] = StandardScaler().fit_transform(x[standardize_cols])

for column in make_dummy_cols:
    dummies = pd.get_dummies(x[column], prefix=column, drop_first=False, dtype=int)
    x = x.drop(column, axis=1)
    for col in dummies.columns:
        x[col] = dummies[col]

C:\Users\Alessio\AppData\Local\Temp\ipykernel_4460\1552330005.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x[col] = dummies[col]
C:\Users\Alessio\AppData\Local\Temp\ipykernel_4460\1552330005.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x[col] = dummies[col]
C:\Users\Alessio\AppData\Local\Temp\ipykernel_4460\1552330005.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

In [125]:
y = LabelEncoder().fit_transform(y)  # encode target categorical variable

In [126]:
x.shape

(76518, 265)

In [127]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [128]:
class TabularDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features.values, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)  # Use long for classification

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [129]:
train_dataset = TabularDataset(x_train, y_train)
test_dataset = TabularDataset(x_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [130]:
class MultiClassNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MultiClassNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

In [131]:
model = MultiClassNN(input_size=x_train.shape[1], num_classes=len(np.unique(y_train)))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/30], Loss: 0.4940
Epoch [2/30], Loss: 0.4589
Epoch [3/30], Loss: 0.4514
Epoch [4/30], Loss: 0.4437
Epoch [5/30], Loss: 0.4384
Epoch [6/30], Loss: 0.4308
Epoch [7/30], Loss: 0.4238
Epoch [8/30], Loss: 0.4165
Epoch [9/30], Loss: 0.4069
Epoch [10/30], Loss: 0.3990
Epoch [11/30], Loss: 0.3888
Epoch [12/30], Loss: 0.3777
Epoch [13/30], Loss: 0.3669
Epoch [14/30], Loss: 0.3549
Epoch [15/30], Loss: 0.3447
Epoch [16/30], Loss: 0.3336
Epoch [17/30], Loss: 0.3191
Epoch [18/30], Loss: 0.3089
Epoch [19/30], Loss: 0.2947
Epoch [20/30], Loss: 0.2828
Epoch [21/30], Loss: 0.2743
Epoch [22/30], Loss: 0.2601
Epoch [23/30], Loss: 0.2492
Epoch [24/30], Loss: 0.2401
Epoch [25/30], Loss: 0.2322
Epoch [26/30], Loss: 0.2235
Epoch [27/30], Loss: 0.2131
Epoch [28/30], Loss: 0.2075
Epoch [29/30], Loss: 0.2021
Epoch [30/30], Loss: 0.1933


In [132]:
model.eval()
correct = 0
total = 0
all_labels = []
all_predictions = []
with torch.no_grad():
    for features, labels in test_loader:
        outputs = model(features)
        probabilities = torch.sigmoid(outputs)  # Apply sigmoid to get probabilities
        predicted = (probabilities > 0.5).float()  # Convert probabilities to binary predictions
        
        total += labels.size(0)
        correct += (predicted.view(-1) == labels.view(-1)).sum().item()  # Compare with original labels

        all_labels.extend(labels.numpy())  # Collect actual labels
        all_predictions.extend(predicted.numpy())  # Collect predicted labels

all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

cm = confusion_matrix(all_labels, all_predictions)

print(f'Accuracy of the model: {100 * correct / total:.2f}%')
print(roc_auc_score(all_labels, all_predictions))
print(classification_report(all_labels, all_predictions))

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

RuntimeError: The size of tensor a (192) must match the size of tensor b (64) at non-singleton dimension 0

In [ ]:
torch.save(model.state_dict(), 'loan_approval.pth')